In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!curl https://sdk.cloud.google.com | bash

In [ ]:
!gcloud init

In [ ]:
!mkdir gcc_temp

In [ ]:
!gsutil cp gs://dhope-input-data/spark/sales.csv.txt ./gcc_temp/.

In [ ]:
!pip install --upgrade google-cloud-bigquery-storage pyarrow

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

In [ ]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DateType

d_schema = StructType([
    StructField("product_id" ,StringType(),True),
    StructField("customer_id" ,StringType(),True),
    StructField("order_date" ,StringType(),True),
    StructField("location" ,StringType(),True),
    StructField("source_order" ,StringType(),True)
]
)

df = spark.read\
    .option("header", "false")\
    .format("csv")\
    .schema(d_schema)\
    .load("./gcc_temp/sales.csv.txt")

df.show()



In [ ]:
import pandas as pd
import pandas_gbq

project_id = "dwhp-437913"
table_id = 'db1.tb_colab_test2'

dfp = df.toPandas()

##if_existsstr replace / fail

pandas_gbq.to_gbq(dfp , table_id, project_id=project_id ,if_exists='append')

In [ ]:
configuration = {
   'query': {
     "useQueryCache": True,
     "use_bqstorage_api": True
   }
}

sql = "select * from db1.tb_colab_test2"

##df2 = pandas_gbq.read_gbq(sql, project_id=project_id)
##
df2 = pandas_gbq.read_gbq(sql, project_id=project_id,configuration=configuration)

df2

In [ ]:
from google.cloud import bigquery

pid = "dwhp-437913"

client = bigquery.Client(pid)

query_str = '''
CALL `dwhp-437913.db1.sp_truncate_test2`();
'''

print (client )

query_job = client.query(query_str)
rows = list(query_job.result())
print(rows)
